<a href="https://colab.research.google.com/github/ohilikeit/Dacon_code_similarity/blob/main/huggingface_CodeBERTa_small_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/code_similarity

/content/drive/MyDrive/code_similarity


In [ ]:
!pip install transformers datasets

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('train_data.csv')
sample_train = pd.read_csv('sample_train.csv')
sum_data = pd.concat([df, sample_train], axis = 0)
sum_data = sum_data(frac=1).reset_index(drop=True, inplace=True)
sum_data.to_csv('train_2.csv')
del df, sample_train, sum_data

In [ ]:
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback, RobertaForMaskedLM, RobertaTokenizer, RobertaForSequenceClassification
import pandas as pd
import numpy as np
from datasets import load_dataset, load_metric
from transformers.utils import logging

MODEL = "huggingface/CodeBERTa-small-v1"
INPUT = "train_2.csv"
MAX_LEN = 512
dataset = load_dataset("csv", data_files=INPUT)['train']
tokenizer = RobertaTokenizer.from_pretrained(MODEL)

#logging.set_verbosity_error() 
logging.set_verbosity_info() 
#--> 두번째에 이거로 교체 후 다시 돌리기 
def example_fn(examples):
    outputs = tokenizer(examples['code1'], examples['code2'], padding=True, max_length=MAX_LEN,truncation=True)
    if 'similar' in examples:
        outputs["labels"] = examples["similar"]
    return outputs

dataset = dataset.map(example_fn, remove_columns=['code1', 'code2', 'similar'])
dataset = dataset.train_test_split(0.1)

_collator = DataCollatorWithPadding(tokenizer=tokenizer)
_metric = load_metric("glue", "sst2")

def compute_metrics(p):
    preds, labels = p
    output =  _metric.compute(references=labels, predictions=np.argmax(preds, axis=-1),)
    return output
    
model = RobertaForSequenceClassification.from_pretrained(MODEL) 

args = TrainingArguments(
    'experience',
    evaluation_strategy = 'steps',
    eval_steps = 500,
    load_best_model_at_end = True,
    metric_for_best_model='accuracy', # validation loss
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay= 0.01,
    do_train=True,
    do_eval=True,
    save_strategy='steps',
    logging_strategy="steps",
    seed=42
)

trainer = Trainer(
        model=model,
        args=args,
        data_collator=_collator,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]

In [65]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 478212
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 44835


Step,Training Loss,Validation Loss,Accuracy
500,0.383300,0.208661,0.922424
1000,0.211700,0.160299,0.940679
1500,0.180600,0.134393,0.949562
2000,0.164200,0.155123,0.940924
2500,0.146400,0.120670,0.954286


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 53135
  Batch size = 32
Saving model checkpoint to experience/checkpoint-500
Configuration saved in experience/checkpoint-500/config.json
Model weights saved in experience/checkpoint-500/pytorch_model.bin
tokenizer config file saved in experience/checkpoint-500/tokenizer_config.json
Special tokens file saved in experience/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num 

Step,Training Loss,Validation Loss,Accuracy
500,0.383300,0.208661,0.922424
1000,0.211700,0.160299,0.940679
1500,0.180600,0.134393,0.949562
2000,0.164200,0.155123,0.940924
2500,0.146400,0.120670,0.954286
3000,0.138400,0.113506,0.958182
3500,0.124700,0.102113,0.963602
4000,0.122500,0.108505,0.963847
4500,0.122500,0.095720,0.965296
5000,0.117200,0.119209,0.958220


Saving model checkpoint to experience/checkpoint-3000
Configuration saved in experience/checkpoint-3000/config.json
Model weights saved in experience/checkpoint-3000/pytorch_model.bin
tokenizer config file saved in experience/checkpoint-3000/tokenizer_config.json
Special tokens file saved in experience/checkpoint-3000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 53135
  Batch size = 32
Saving model checkpoint to experience/checkpoint-3500
Configuration saved in experience/checkpoint-3500/config.json
Model weights saved in experience/checkpoint-3500/pytorch_model.bin
tokenizer config file saved in experience/checkpoint-3500/tokenizer_config.json
Special tokens file saved in experience/chec

TrainOutput(global_step=15000, training_loss=0.11709304682413738, metrics={'train_runtime': 48404.3445, 'train_samples_per_second': 29.639, 'train_steps_per_second': 0.926, 'total_flos': 6.353305335475046e+16, 'train_loss': 0.11709304682413738, 'epoch': 1.0})

In [64]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

88

In [71]:
import pandas as pd
logging.set_verbosity_error() 
TEST = "test.csv"
SUB = "sample_submission.csv"

test_dataset = load_dataset("csv", data_files=TEST)['train']
test_dataset = test_dataset.map(example_fn, remove_columns=['code1', 'code2'])

predictions = trainer.predict(test_dataset)

df = pd.read_csv(SUB)
df['similar'] = np.argmax(predictions.predictions, axis=-1)
df.to_csv('experience_submit.csv', index=False)
df

0         0
1         1
2         0
3         0
4         0
         ..
179695    1
179696    1
179697    0
179698    1
179699    1
Name: similar, Length: 179700, dtype: int64